In [1]:
from hexarena.env import ForagingEnv

env = ForagingEnv()

In [2]:
observation, _ = env.reset()

obss = []
for _ in range(50):
    observation, *_ = env.step(env.action_space.sample())
    obss.append([*observation['monkey'], *observation['color'], observation['rewarded']])

In [3]:
env.observation_space

Dict('color': Box(-inf, inf, (2,), float32), 'monkey': MultiDiscrete([19 19]), 'rewarded': Discrete(2))

In [4]:
env.state_space['box_0']

Dict('cue': Box(0.0, 1.0, (1,), float32), 'food': Discrete(2))

In [6]:
from stable_baselines3 import PPO
from sb3_contrib import RecurrentPPO
from jarvis.utils import tqdm
from irc.utils import ProgressBarCallback

n_steps = 200 # time steps per update in PPO
batch_size = 100 # batch size in PPO

algo = RecurrentPPO(
    policy='MultiInputPolicy', env=env,
    n_steps=n_steps, batch_size=batch_size,
)

n_updates = 100
algo.policy.set_training_mode(True)
with tqdm(total=n_updates, unit='update', leave=False) as pbar:
    pbar_cb = ProgressBarCallback(pbar, disp_freq=n_steps)
    # pbar_cb.logs = manager.logs
    # if len(manager.logs)>0:
    #     pbar_cb.reward, pbar_cb.food = manager.logs[-1]
    algo.env.envs[0].needs_reset = False
    algo.learn(
        total_timesteps=n_updates*n_steps,
        callback=pbar_cb, reset_num_timesteps=False,
    )
algo.policy.set_training_mode(False)

ValueError: Policy MultiInputPolicy unknown